Example of looping through all comments, and storing info in a dataframe

In [65]:
#Grab packages

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [66]:
#Connect to webpage

options = webdriver.ChromeOptions()
#options.add_argument('--headless')


options.binary_location = "/usr/bin/google-chrome"
chrome_driver_binary = "/usr/local/bin/chromedriver"
#options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
#chrome_driver_binary = "C:/Users/Alek/chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)



/home/ravisolter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [67]:
# Plan for scraping archives

#    - create a list of archive URLs

archive_periods = []

for j in range(2007,2021):
    for k in range(1,12):
        single_period = 'https://www.blackwhitereadallover.com/archives/'+str(j)+'/'+str(k)
        archive_periods.append(single_period)
        ;
        
# Subsetting out some periods for which there are no articles
active_archives = archive_periods[1:len(archive_periods)-8]
        

In [64]:
driver.get('https://www.blackwhitereadallover.com/archives/2007/2')
for a in driver.find_elements_by_xpath('.//span[contains(text(), "data-analytics-link")]/a'):
    print(a.get_attribute('href'));
    
   
    

In [89]:
#    - Visit earch URL and check to see if the "load more" button exists (will when articles > 30)
#    - collect the article URLS from each archive page


# visiting a test page

for u in range(0,1):
    #print(u)
    driver.get(active_archives[u])
    
    # Retrieving number of articles from title 
    archive_title = driver.find_element_by_xpath('/html/body/div[1]/div[2]/h1')
    #print(archive_title.text)
    
    start = archive_title.text.find('(')+1
    start = archive_title.text[start:]
    articles = int(start.replace(')',''))
    #print(articles)
    
    
    # Retrieving the article links themselves
    for q in range(0,articles):
        
        article_link = driver.find_elements_by_css_selector('h2.c-entry-box--compact__title>a')
        article_link = article_link[q].get_attribute('href')
        
        print(article_link) 
           
    ;



https://www.blackwhitereadallover.com/2007/2/28/2836773/february-27-afternoon-quick-notes
https://www.blackwhitereadallover.com/2007/2/27/2836772/juventus-modena-wrap-up-leaving-it-late
https://www.blackwhitereadallover.com/2007/2/23/2836771/thursday-feb-22-quick-notes-crybabies-politics-cocaine-lindsay-lohan
https://www.blackwhitereadallover.com/2007/2/22/2836770/weekend-wrap-up-a-little-tardy
https://www.blackwhitereadallover.com/2007/2/16/2836769/stop-the-presses-tudor-stays-and-other-junk
https://www.blackwhitereadallover.com/2007/2/15/2836768/february-15-2007-quick-notes
https://www.blackwhitereadallover.com/2007/2/15/2836767/primavera-player-profile-sebastian-giovinco
https://www.blackwhitereadallover.com/2007/2/14/2836766/march-12-2003-igor-tudor-day


In [ ]:
driver.quit() 

In [5]:
# Have selenium wait until the BWRAO logo loads (therefore, has logged in properly)
element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-global-header__logo-large"))
    )




In [6]:

### Insert loop here to visit relevant URLs

driver.get("https://www.blackwhitereadallover.com/2020/4/3/21206623/miralem-pjanic-marco-verratti-juventus-paris-saint-germain-2020-serie-a-summer-transfer-rumors#comments")

## Grab article title, author

blog_title = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[2]/h1')
print(blog_title.text)

article_author = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[3]/span[1]/span[1]/a/span')
print(article_author.text)

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-comments__list"))
    )

#Find number of comments

n_comments = driver.find_element_by_xpath('//*[@id="comments"]/div[1]/span/span')
n_comments = int(n_comments.text)


Report: PSG turns down Miralem Pjanic-Marco Verratti swap deal
Danny Penza


In [7]:
# Loop through all comments, taking title, content, and meta data (recs, author, etc)

from selenium.common.exceptions import NoSuchElementException

comments_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    xpath = '//*[@id="comments"]/div[2]/div['+str(i)+']'
     
    comment_title = driver.find_element_by_xpath(xpath + '/div[1]')
    comment_body = driver.find_element_by_xpath(xpath + '/div[2]')
    
    
    if driver.find_elements_by_xpath(xpath + '/div[4]'):
        poster = driver.find_element_by_xpath(xpath + '/div[4]/span[1]/a[1]')
        post_date = driver.find_element_by_xpath(xpath + '/div[4]/span[1]/a[2]')
        
        if driver.find_elements_by_xpath(xpath + '/div[4]/span[2]/button[2]/span[2]'):
            recs = driver.find_element_by_xpath(xpath + '/div[4]/span[2]/button[2]/span[2]')
            recs = int(recs.text.replace('(','').replace(')',''))
        else:
            recs = 0
    else:
        poster = driver.find_element_by_xpath(xpath + '/div[3]/span[1]/a[1]')
        post_date = driver.find_element_by_xpath(xpath + '/div[3]/span[1]/a[2]')
        
        if driver.find_elements_by_xpath(xpath + '/div[3]/span[2]/button[2]/span[2]'):
            recs = driver.find_element_by_xpath(xpath + '/div[3]/span[2]/button[2]/span[2]')
            recs = int(recs.text.replace('(','').replace(')',''))
        else:
            recs = 0

    comment = pd.DataFrame([comment_title.text,comment_body.text,poster.text,post_date.text.split('|')[0],
                            post_date.text.split('|')[1],recs])
    comments_df = pd.concat([comments_df,comment.T])
    
    ;
   

In [8]:
comments_df.columns = ['Comment Title','Comment Body','Poster','Date','Time','Recs']

# Adding article title and author to comments df
comments_df.insert(0, 'Article', blog_title.text)
comments_df.insert(1, 'Author', article_author.text)

comments_df.head()



,Article,Author,Comment Title,Comment Body,Poster,Date,Time,Recs
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,interesting video about how some stats should/...,resko,"Apr 3, 2020",6:37 AM,0
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,Well that was a tough decision for PSG!,Paolo C,"Apr 3, 2020",6:42 AM,8
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,hmmm…………maybe if we added Khedira……….or Bernar...,Ricardo 1,"Apr 3, 2020",10:44 AM,0
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,Interest chart here. Various strikers over/und...,Paolo C,"Apr 3, 2020",9:59 AM,6
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,,Dybala is an amazing finisher\nso much this,#finoallafine,"Apr 5, 2020",11:50 PM,0


In [36]:
comments_df.to_csv('comments.csv', index=False)


In [40]:
driver.quit()